In [24]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [25]:
# Hyper parameters
num_epochs: int = 30
batch_size: int  = 100
learning_rate: float = 0.001

# Image pre-processing
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

# CIFAR-10 dataset 
train_dataset = torchvision.datasets.CIFAR10(root='./cifar10_dataset',
                                             train=True,
                                             transform=transform,
                                             download=True)

# Why are all the image preprocessing transforms not applied to test set?
test_dataset = torchvision.datasets.CIFAR10(root='./cifar10_dataset',
                                           train=False,
                                           transform=transforms.ToTensor())

# Data Loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

Files already downloaded and verified


In [26]:
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsampling = None):
        super(ResidualBlock, self).__init__()
        # Possible downsampling layer
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        
        #Regular conv layer
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # downsampling layer to downsample input before adding if necessary
        self.downsampling = downsampling
    
    def forward(self, x):
        # Save input so it can be added later
        residual = x.clone()
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsampling:
            residual = self.downsampling(x)
        out += residual 
        out = self.relu(out)
        return out

In [27]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        """
        layers will be a list of number of blocks per layer. Ex: [2, 2, 2] indicates 3 layers with 2 residual blocks each.
        """
        super(ResNet, self).__init__()
        self.in_channels = 16
        # color image, 3 channels, 32x32
        self.conv = conv3x3(3,16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)

        # Layer - each layer has a couple of residual blocks
        self.layer1 = self.make_layer(block, 16, layers[0], stride = 1) # No downsampling in layer1
        self.layer2 = self.make_layer(block, 32, layers[1], stride = 2)
        self.layer3 = self.make_layer(block, 64, layers[2], stride = 2)
        
        # 32x32 orig. 2 downsamples of stride 2. So 32x32 -> 16x16 -> 8x8
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
    
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsampling = None

        if (self.in_channels != out_channels) or (stride != 1):
            downsampling = nn.Sequential(conv3x3(self.in_channels, out_channels, stride), 
                                         nn.BatchNorm2d(out_channels))
            
        residual_blocks = []
        residual_blocks.append(block(self.in_channels, out_channels, stride = stride, downsampling = downsampling))
        # change the class var to match the new value which is outchannels
        self.in_channels = out_channels
        residual_blocks.append(block(self.in_channels, out_channels))
        return nn.Sequential(*residual_blocks)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        # Resize before passing to the classification layer
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out 


In [28]:
model = ResNet(ResidualBlock, [2,2,2])
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [29]:
# Train
decay = 0
model.train()
for epoch in range(num_epochs):
    
    # Decay learning rate by a factor of 0.5 every 20 epochs
    if (epoch+1) % 20 == 0:
        decay += 1
        optimizer.param_groups[0]['lr'] = learning_rate * (0.5**decay)
        print(f"The new learning rate is {optimizer.param_groups[0]['lr']}")
    
    for i, (images, labels) in enumerate(train_loader):
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}] Loss: {loss.item():.4f}")

Epoch [1/30], Step [100/500] Loss: 1.6425
Epoch [1/30], Step [200/500] Loss: 1.3677
Epoch [1/30], Step [300/500] Loss: 1.1137
Epoch [1/30], Step [400/500] Loss: 1.1006
Epoch [1/30], Step [500/500] Loss: 1.0586
Epoch [2/30], Step [100/500] Loss: 1.0897
Epoch [2/30], Step [200/500] Loss: 0.9055
Epoch [2/30], Step [300/500] Loss: 1.0513
Epoch [2/30], Step [400/500] Loss: 0.6971
Epoch [2/30], Step [500/500] Loss: 0.9806
Epoch [3/30], Step [100/500] Loss: 0.7913
Epoch [3/30], Step [200/500] Loss: 0.8852
Epoch [3/30], Step [300/500] Loss: 0.9440
Epoch [3/30], Step [400/500] Loss: 0.7734
Epoch [3/30], Step [500/500] Loss: 0.7047
Epoch [4/30], Step [100/500] Loss: 0.6461
Epoch [4/30], Step [200/500] Loss: 0.5150
Epoch [4/30], Step [300/500] Loss: 0.7658
Epoch [4/30], Step [400/500] Loss: 0.4808
Epoch [4/30], Step [500/500] Loss: 0.6588
Epoch [5/30], Step [100/500] Loss: 0.7954
Epoch [5/30], Step [200/500] Loss: 0.6534
Epoch [5/30], Step [300/500] Loss: 0.5698
Epoch [5/30], Step [400/500] Loss:

In [30]:
# Test 
model.eval()
with torch.no_grad(): # Ensure no gradient computation. 
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print(f"Accuracy of the model on test images: {100*correct/total}")

Accuracy of the model on test images: 86.04
